# Merge data for Gauthier

In [ ]:
# Import modules
import pandas as pd

def add_group_names(institution, df_groups, save=False):
    # Get Researchers
    df = pd.read_csv(f'../data/20220309/nodes_{institution}_20220309.csv', converters={'groups':eval})
    
    # Add group names
    groups_dict = {}
    
    def get_group_names(lst, df_groups):
        groups_names = []
        for group_id in lst:
            group_name = df_groups.loc[df_groups['url_id']==group_id, 'name'].values[0]
            groups_names.append(group_name)
            groups_dict[group_id] = group_name
        return groups_names
    # Merge
    df['groups_names'] = df['groups'].apply(lambda x: get_group_names(x, df_groups))
    
    groups_inst = pd.DataFrame.from_dict(groups_dict, orient='index').reset_index()
    groups_inst.columns=['url_id', 'name']
    groups_df = groups_inst.merge(df_groups, how='left', on='url_id', suffixes=['','_y'])
    groups_df = groups_df.drop(columns=['name_y'])
    
    if save:
        df.to_csv(f'../data/20220309/nodes_{institution}_withgroups_20220309.csv', index=None)
        groups_df.to_csv(f'../data/20220309/groups_{institution}_20220309.csv', index=None)
    
    return df, groups_df

def get_unique_groups(df, institution, save=False):
    unique_groups = set(df['groups_names'].sum())
    if save:
        unique_groups_df = pd.DataFrame(unique_groups)
        unique_groups_df.to_csv(f'../data/20220309/groups_unique_{institution}_20220309.csv', index=None)
    return unique_groups

In [ ]:
# Get Groups
df_groups = pd.read_csv('../data/20220314/group_data_20220314.csv')
df_groups['url_id'] = '/' + df_groups['url_id']

institution_list = ['UVic-UCC', 'UB']
institution_list = ['UPC_CIMNE']
# institution_list = ['UVic-UCC']
# institution_list = ['UB']

for institution in institution_list:
    add_group_names(institution, df_groups, save=True)